# Day 2: APIs and Data Exploration


---

# <font color='red'> Please make your own copy of this notebook by selecting "File" then "Save a Copy in Drive"

<a id="learning-objectives"></a>
## Learning Objectives
*After completing this notebook, you will be able to:*

- Read API documentation
- Make API requests
- Clean, filter and sort large datasets
- Perform an exploratory data analysis 

## <font color='blue'> Using the Police API in Python
    
Let’s finally find out about crimes in different parts of the UK, using the Police API!

The documentation for the ‘street level crimes’ section is here: https://data.police.uk/docs/method/crime-street/

We already know how to make this demo API request in our web browsers- we simply click on this URL...
https://data.police.uk/api/crimes-street/all-crime?lat=52.629729&lng=-1.131592&date=2022-01 

But how can we make the same API request in Python? After all, that's much more efficient, repeatable and scalable. 

The first thing we need to do is import the **library** that will enable us to make API requests. The name of this library is **requests**

In [ ]:
import requests

Now, let's make a GET request to the demo URL that we visited in our web browsers.

We do this using the `get` function from the `requests` library.

The notation `requests.get()` shows us that the `get` command comes from the `requests` library.

In [ ]:
crime_request = requests.get('https://data.police.uk/api/crimes-street/all-crime?lat=52.629729&lng=-1.131592&date=2022-01')


Now let's inspect the variable we've just created, `crime_request`. You might expect it to look like JSON data...

In [ ]:
print(crime_request)

But it's not! The variable `crime_request` actually contains a few different pieces of information.

Think of `crime_request` as being like a package that arrives from Amazon. It contains some packaging, some plastic wrapping, some bubblewrap, but the thing you're interested in is the product you ordered.

The thing we're interested in is the JSON data, which is packaged up inside the `crime_request` variable.

We can extract the JSON data as follows:

In [ ]:
crime_request.json()

It makes more sense to store the JSON data in its own variable, so we can analyse it further.

In [ ]:
crime_data = crime_request.json()

As a final trick, we can convert our JSON data into something called a `DataFrame`. 

We'll learn more about these very soon, but for now- a `DataFrame` is a way of storing data in a table in Python.

It's much easier to work with data in `DataFrame` format than in JSON format, because `DataFrames` are much more user friendly, and there are lots of commands in Python that can be used to manipulate, sort, filter and clean them. 

We'll use the `pandas` library for this.

In [ ]:
import pandas as pd

The next cell transforms our JSON data into a `DataFrame`

In [ ]:
crime_dataframe = pd.DataFrame(crime_data)

We can preview what our crime `DataFrame` looks like...

In [ ]:
crime_dataframe.head()

We can get the total number of rows, or crimes, in the `DataFrame`

In [ ]:
len(crime_dataframe)

And we can finally get a breakdown of different crime types.

In [ ]:
crime_dataframe['category'].value_counts()

## <font color='red'> Exercise: The final Police data challenge

Now let’s bring all our skills together. In your notebooks:

* Pick a location in the UK of your choice
* Find out how many crimes were reported in total in June 2022 in that area
* Get a breakdown of the number of crimes by type
* Be ready to share your chosen location and most common type of crime with the rest of the class!


<a id="dataframe-methods"></a>

# <font color='blue'> Introducing Dataframes

A DataFrame is the most important class of object in pandas. Let's explore how they work!
    
First, we'll need to import the `pandas` library so we can start using commands from it. We import the library and give it a shorter name `pd` for convenience. 
    
This means whenever we want to use a function from `pandas`, we don't need to type `pandas.read_csv()` for example- we can type `pd.read_csv()` instead which is shorter, and is the smart but lazy approach!

In [ ]:
import pandas as pd

Now, we'll read in our gender pay gap dataset using the `read_csv` function. 

In [ ]:
pay_gap = pd.read_csv('UK Gender Pay Gap Data - 2021 to 2022.csv')

It's easy to preview the first few rows of a `DataFrame`

In [ ]:
pay_gap.head()

The ``shape`` attribute is a good way of figuring out how big our dataset is

In [ ]:
pay_gap.shape

And the `columns` attribute shows us all the column names in our dataset.

In [ ]:
pay_gap.columns

We can check the type of data in each column too.

In [ ]:
pay_gap.dtypes

**Can you identify any data quality issues that we might need to handle later?**

# <font color='blue'> Missing values

Let's check our data for missing values.

We can do this using the `isnull()` method and summing up the values for each column.

In [ ]:
pay_gap.isnull().sum()

We can choose to drop rows containing ``NaN`` values, or fill in ``NaN`` values with a string, float or other element of our choice. 

Be careful when doing either of these things; you could end up unintentionally removing rows, or filling in values that don't make sense or aren't accurate.

In this case, it would be important to clarify whether a ``NaN`` value in a particular column means the amount is zero, or whether it means the amount is unknown.

We can **fill in** NaN values with a value of our choice using `fillna()`. For example, it makes sense to fill in `CompanyLinkToGPGInfo` with a string like 'no URL provided'.

In [ ]:
pay_gap['CompanyLinkToGPGInfo'].fillna('No URL',inplace=True)

We can now see that this column no longer has any missing values.

In [ ]:
pay_gap.isnull().sum()

We might want to **drop** rows where there is no company number provided, since this means we won't be able to look up the company on Companies House.

In [ ]:
pay_gap.dropna(subset=['CompanyNumber'],inplace=True)

Again, we can now see that there are no missing values in the `CompanyNumber` column.

In [ ]:
pay_gap.isnull().sum()

<a id="selecting-cols"></a>

# <font color='blue'> Selecting columns

Pandas DataFrames have structural similarities with Python-style lists and dictionaries. We can select, or extract, columns from a `DataFrame` using column names.



In the example below, we select a column of data using the name of the column in a similar manner to how we select a dictionary value with the dictionary key.

In [ ]:
pay_gap['Address']

We can select multiple columns using this syntax

In [ ]:
pay_gap[['EmployerName','Address','CompanyNumber']]

<a id="transforming-cols"></a>

# <font color='blue'> Transforming columns
    
Once we've selected columns, we can perform transformations on them or calculations with them, in a similar way to how we perform calculations in base Python.

In [ ]:
pay_gap['DiffMeanBonusPercent']/100

We can also apply functons to columns that contain strings.

In [ ]:
pay_gap['EmployerName'].str.lower()

We can replace strings. This can be used to remove strings, too by replacing them with a blank space or `''`

In [ ]:
pay_gap['EmployerName'].str.lower().str.replace(' ','-').str.upper()

---
## <font color='red'> Exercise: Data cleaning and calculations
    
Let's start to clean up our dataset and answer the questions in our analysis.

Create a new column in the DataFrame, called `DiffMedianBonusProportion` which is calculated by dividing the `DiffMedianBonusPercent` column by 100

In [ ]:
pay_gap['DiffMedianBonusProportion'] = pay_gap['DiffMedianBonusPercent']/100

In [ ]:
pay_gap.head()

You might have noticed earlier when running the `dtypes` command that the `DiffMeanHourlyPercent` and `DiffMedianHourlyPercent` columns are currently being treated like strings. We need to convert them into numeric columns so we can perform calculations with them.

Start by removing the % sign in both of these columns (hint: you can use `str.replace` and replace the % symbol with `''`. You will also need to overwrite the original columns in your dataset with the new ones using syntax similar to this: `df[column_name] = df[column_name]*100`.)

In [ ]:
pay_gap['DiffMeanHourlyPercent'] = pay_gap['DiffMeanHourlyPercent'].str.replace('%','')
pay_gap['DiffMedianHourlyPercent'] = pay_gap['DiffMedianHourlyPercent'].str.replace('%','')
pay_gap.head()

Now, try running this `pd.to_numeric` function to convert both of these columns into numeric ones:

In [ ]:
pay_gap['DiffMeanHourlyPercent'] = pd.to_numeric(pay_gap['DiffMeanHourlyPercent'])
pay_gap['DiffMedianHourlyPercent'] = pd.to_numeric(pay_gap['DiffMedianHourlyPercent'])

In [ ]:
pay_gap.dtypes

Can you figure out how to use the `.max()` method to find the largest value of `DiffMeanHourlyPercent` in the dataset?

In [ ]:
pay_gap['DiffMeanHourlyPercent'].min()

One of our aims is to find the average pay gap for all companies in the dataset. Decide on:

* Which column you will use as your measure of pay gap- there are several different ones available
* Whether you will use the mean or median to calculate the average

Be ready to justify and explain your choices.

Finally, work out the average pay gap across the whole dataset using the column of your choice, and your choice of the `.mean()` or `.median()` function.

In [ ]:
pay_gap['DiffMedianHourlyPercent'].median()

---
## <font color='green'> Solutions: Data cleaning and calculations
    
Let's start to clean up our dataset and answer the questions in our analysis.

Create a new column in the DataFrame, called `DiffMedianBonusProportion` which is calculated by dividing the `DiffMedianBonusPercent` column by 100

In [ ]:
pay_gap['DiffMedianBonusProportion'] = pay_gap['DiffMedianBonusPercent']/100

2. You might have noticed earlier when running the `dtypes` command that the `DiffMeanHourlyPercent` and `DiffMedianHourlyPercent` columns are currently being treated like strings. We need to convert them into numeric columns so we can perform calculations with them.

Start by removing the % sign in both of these columns (hint: you can use `str.replace` and replace the % symbol with `''`. You will also need to overwrite the original columns in your dataset with the new ones using syntax similar to this: `df[column_name] = df[column_name]*100`.)

In [ ]:
pay_gap['DiffMeanHourlyPercent'] = pay_gap['DiffMeanHourlyPercent'].str.replace('%','')
pay_gap['DiffMedianHourlyPercent'] = pay_gap['DiffMedianHourlyPercent'].str.replace('%','')

Now, try running this `pd.to_numeric` function to convert both of these columns into numeric ones:

In [ ]:
pay_gap['DiffMeanHourlyPercent'] = pd.to_numeric(pay_gap['DiffMeanHourlyPercent'])
pay_gap['DiffMedianHourlyPercent'] = pd.to_numeric(pay_gap['DiffMedianHourlyPercent'])

3. Can you figure out how to use the `.max()` method to find the largest value of `DiffMeanHourlyPercent` in the dataset?

In [ ]:
pay_gap['DiffMedianHourlyPercent'].max()

One of our aims is to find the average pay gap for all companies in the dataset. Decide on:

* Which column you will use as your measure of pay gap- there are several different ones available
* Whether you will use the mean or median to calculate the average

Be ready to justify and explain your choices.

Finally, work out the average pay gap across the whole dataset using the column of your choice, and your choice of the `.mean()` or `.median()` function.

In [ ]:
pay_gap['DiffMedianHourlyPercent'].median()

-----

The `describe()` method is a handy way of accessing summary statistics for an entire `DataFrame` in one go.

In [ ]:
pay_gap.describe()

<a id="selecting-rows"></a>

# <font color='blue'> Selecting rows

## Selecting rows using logical tests

Imagine we want to find all companies where the value of `DiffMedianHourlyPercent` was bigger than 50%.

We start by writing a **filter** or a logical test that will be `True` for the rows we're interested in. 

We're interested in the `DiffMedianHourlyPercent` column so our filter looks like this:

In [ ]:
pay_gap['DiffMedianHourlyPercent']>50

When we inspect this filter, we can see it's a long list of `True` and `False` values; the value of the filter is `True` for rows that pass the logical test and `False` for rows that don't.

Now we **apply** our filter to our DataFrame

In [ ]:
pay_gap[pay_gap['DiffMedianHourlyPercent']>50]

We can also write this a bit more neatly, and create a new `DataFrame` from the result of our filtering operation.

In [ ]:
pay_gap_filter = pay_gap['DiffMedianHourlyPercent']>50
large_pay_gap_companies = pay_gap[pay_gap_filter]

In [ ]:
large_pay_gap_companies.describe()

It's also possible to combine logical tests using `and` and `or` operators. For example, to find all rows where `DiffMedianHourlyPercent` is greater than 50% **and** `DiffMeanHourlyPercent` is greater than 50%, we can write:

**Note that the `and` operator here is written as `&`**

In [ ]:
pay_gap_filter_2 = (pay_gap['DiffMedianHourlyPercent']>50) & (pay_gap['DiffMeanHourlyPercent']>50)
pay_gap[pay_gap_filter_2]

Similarly, to find all rows where `Profit` is greater than \$100 **and** `Sale Price` is greater than \$200, we can write:

**Note that the `or` operator here is written as `|`**

In [ ]:
pay_gap_filter_3 = (pay_gap['DiffMedianHourlyPercent']>50) | (pay_gap['DiffMeanHourlyPercent']>50)
pay_gap[pay_gap_filter_3]

We can also use the ``str.contains()`` method to find all rows that contain a particular string.

In [ ]:
pay_gap['Address'].fillna('No address provided',inplace=True)

In [ ]:
address_filter = pay_gap['Address'].str.lower().str.contains('london')
pay_gap[address_filter]

---
## <font color='red'> Exercise: Filtering rows and columns

Approximately how many companies in the dataset are in London?
How many are in Birmingham?

In [ ]:
london_filter = pay_gap['Address'].str.lower().str.contains('london')
london_companies = pay_gap[london_filter]
london_companies.shape[0]

In [ ]:
birmigham_filter = pay_gap['Address'].str.lower().str.contains('birmingham')
birmigham_companies = pay_gap[birmigham_filter]
birmigham_companies.shape[0]

 Is there a difference between the average pay gap for companies in London vs Birmingham?

In [ ]:
london_companies['DiffMedianHourlyPercent'].median()

In [ ]:
birmigham_companies['DiffMedianHourlyPercent'].median()

Approximately how many of the organisations in the dataset are schools? What is their average pay gap?

In [ ]:
school_logical_test = pay_gap['EmployerName'].str.lower().str.contains('school')
schools = pay_gap[school_logical_test]
schools['DiffMedianHourlyPercent'].median()

---
## <font color='green'> Solutions: Filtering rows and columns

Approximately how many companies in the dataset are in London?
How many are in Birmingham?

In [ ]:
companies_in_london = pay_gap[pay_gap['Address'].str.lower().str.contains('london')]
companies_in_birmingham = pay_gap[pay_gap['Address'].str.lower().str.contains('birmingham')]

print(companies_in_london.shape)
print(companies_in_birmingham.shape)

 Is there a difference between the average pay gap for companies in London vs Birmingham?

In [ ]:
print(companies_in_london['DiffMedianHourlyPercent'].median())
print(companies_in_birmingham['DiffMedianHourlyPercent'].median())

Approximately how many of the organisations in the dataset are schools? What is their average pay gap?

In [ ]:
pay_gap[pay_gap['EmployerName'].str.lower().str.contains('school')]['DiffMedianHourlyPercent'].median()

<a id="value-counts"></a>
# <font color='blue'> Getting value counts

Sometimes we might want to see the breakdown of different values in a column. This is easy with the `value_counts` function.

In [ ]:
pay_gap['EmployerSize']

In [ ]:
pay_gap['EmployerSize'].value_counts()

In [ ]:
pay_gap['EmployerSize'].value_counts('normalize')

<a id="groupby"></a>
# <font color='blue'> Grouping data

Sometimes we might want a more detailed breakdown using more than one column. 

Let's look at the mean sale price for each brand.

In [ ]:
pay_gap.groupby('EmployerSize')['DiffMeanBonusPercent'].median()

In [ ]:
pay_gap.groupby('EmployerSize')['DiffMeanBonusPercent'].mean()

In [ ]:
pay_gap.groupby('EmployerSize')['DiffMeanBonusPercent'].count()

---
## <font color='red'> Exercise: Grouping
    
Is there a relationship between the size of a company and its pay gap? 

<a id="sorting-data"></a>

# <font color='blue'> Sorting data
    
It's easy to sort data in ascending/descending order according to a particular column. We do this using the `sort_values` method.

In [ ]:
pay_gap.sort_values(by=['DiffMedianHourlyPercent'],ascending=False)

---
## <font color='red'> Exercise: The final pay gap challenge

Can you find the top 5 companies with the largest pay gaps:
    
* In favour of men
* In favour of women

Do you think your results are meaningful? 

How could you filter your data _before_ sorting, to solve this problem? Calculate the new top 5 companies with the largest pay gaps in favour of men/women using your solution.